In [921]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split,Dataset,Subset
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import optuna

In [922]:
cuda = True if torch.cuda.is_available() else False

In [923]:
train=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/train.csv")
test=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/test.csv")

In [924]:
class FADataset(Dataset):
    def __init__(self, data,train=True):
        super().__init__()
        if(train):
            self.x=data.drop(['id','y'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            self.y=torch.tensor(data['y'].values,dtype=torch.float32)
        else:
            self.x=data.drop(['id'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data=self.x[idx]
        target=self.y[idx]
        return data,target,idx


In [925]:
dataset=FADataset(train,train=True)
trainset, validset = random_split(dataset, [0.9, 0.1])
testset=FADataset(test,train=False)
test_dl=DataLoader(testset,shuffle=False, batch_size=80)

In [934]:
class MLP(nn.Module):
    def __init__(self,params):
        super().__init__()
        self.model=nn.Sequential(nn.Linear(2,params['num_hidden1']),
                                nn.Linear(params['num_hidden1'],params['num_hidden2']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden2'],params['num_hidden3']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden3'],params['num_hidden4']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden4'],params['num_hidden5']),
                                nn.ReLU(True),  
                                nn.Linear(params['num_hidden5'],params['num_hidden6']),
                                nn.Linear(params['num_hidden6'],1)
                                )
    def forward(self, x):
        return self.model(x)


In [935]:
def train_and_evaluate(params,model,datadl):

    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    

    criterion = nn.MSELoss()

    losses=[]
    num_epochs=50
    model.train()
    for epoch in range(num_epochs):
        for batch,(data, target,idx) in enumerate(datadl):
            outputs = model(data)
            loss = criterion(outputs.reshape(-1), target.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    model.eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(datadl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)
    
    return model,loss


In [947]:

def objective(trial):
        
        params = {

            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
            'batch_size': trial.suggest_int("batch_size", 64, 320,step=16),
            'num_hidden1': trial.suggest_int("num_hidden1", 50, 500),
            'num_hidden2': trial.suggest_int("num_hidden2", 50, 600),
            'num_hidden3': trial.suggest_int("num_hidden3", 50, 600),
            'num_hidden4': trial.suggest_int("num_hidden4", 50, 600),
            'num_hidden5': trial.suggest_int("num_hidden5", 50, 600),
            'num_hidden6': trial.suggest_int("num_hidden6", 50, 600)
            }
        

        train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
        valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])


        model= MLP(params)
        best_model,loss=train_and_evaluate(params,model,train_dl)
      
        torch.save(best_model.state_dict(), f"model_0_trial_{trial.number}.pth")
    
        return loss
    

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-05-02 19:35:42,954] A new study created in memory with name: no-name-a0c46c65-d09c-48aa-941e-0262404756c6
[I 2023-05-02 19:36:03,994] Trial 0 finished with value: 0.02784346676328116 and parameters: {'learning_rate': 0.0439737744752616, 'optimizer': 'SGD', 'batch_size': 80, 'num_hidden1': 445, 'num_hidden2': 590, 'num_hidden3': 471, 'num_hidden4': 328, 'num_hidden5': 518, 'num_hidden6': 248}. Best is trial 0 with value: 0.02784346676328116.


loss: 0.027843 


[I 2023-05-02 19:36:31,592] Trial 1 finished with value: 0.17537788725816286 and parameters: {'learning_rate': 0.05861657070288599, 'optimizer': 'Adam', 'batch_size': 112, 'num_hidden1': 179, 'num_hidden2': 369, 'num_hidden3': 474, 'num_hidden4': 137, 'num_hidden5': 315, 'num_hidden6': 344}. Best is trial 0 with value: 0.02784346676328116.


loss: 0.175378 


[I 2023-05-02 19:37:04,340] Trial 2 finished with value: 3.5431608645121258 and parameters: {'learning_rate': 0.07433492974991665, 'optimizer': 'Adam', 'batch_size': 96, 'num_hidden1': 314, 'num_hidden2': 163, 'num_hidden3': 525, 'num_hidden4': 443, 'num_hidden5': 204, 'num_hidden6': 192}. Best is trial 0 with value: 0.02784346676328116.


...

loss: 0.054000 


[I 2023-05-02 19:41:32,791] Trial 28 finished with value: 0.14545544112722078 and parameters: {'learning_rate': 0.022261712534747437, 'optimizer': 'Adam', 'batch_size': 304, 'num_hidden1': 429, 'num_hidden2': 191, 'num_hidden3': 383, 'num_hidden4': 99, 'num_hidden5': 278, 'num_hidden6': 288}. Best is trial 11 with value: 0.022742727609432262.


loss: 0.145455 


[I 2023-05-02 19:41:40,554] Trial 29 finished with value: 0.10375757763783137 and parameters: {'learning_rate': 0.01809282833988313, 'optimizer': 'SGD', 'batch_size': 224, 'num_hidden1': 457, 'num_hidden2': 124, 'num_hidden3': 209, 'num_hidden4': 291, 'num_hidden5': 503, 'num_hidden6': 163}. Best is trial 11 with value: 0.022742727609432262.


loss: 0.103758 


In [948]:
model_0= MLP(study.best_params)
model_0.load_state_dict(torch.load("C:/Users/MJ/Desktop/FunctionApproximation/model_0_trial_11.pth"))
model_0.eval()
losses=[]
for batch,(data, target,idx) in enumerate(valid_dl):
    outputs = model_0(data)
    loss = criterion(outputs.reshape(-1), target.reshape(-1))
    losses.append(loss.item())
print(f"loss: {sum(losses)/len(losses):>7f} ")
loss=sum(losses)/len(losses)



loss: 0.024060 


In [950]:
second_params={'learning_rate': 0.0008280625643997466, 
                'optimizer': 'Adam', 
                'batch_size': 320, 
                'num_hidden1': 387, 
                'num_hidden2': 262, 
                'num_hidden3': 79, 
                'num_hidden4': 132, 
                'num_hidden5': 592, 
                'num_hidden6': 70}

model_1= MLP(second_params)
model_1.load_state_dict(torch.load("C:/Users/MJ/Desktop/FunctionApproximation/model_0_trial_10.pth"))
model_1.eval()
losses=[]
for batch,(data, target,idx) in enumerate(valid_dl):
    outputs = model_1(data)
    loss = criterion(outputs.reshape(-1), target.reshape(-1))
    losses.append(loss.item())
print(f"loss: {sum(losses)/len(losses):>7f} ")
loss=sum(losses)/len(losses)


loss: 0.023443 


In [956]:
def objective_en(trial):
    a=trial.suggest_float('a',0, 10)
    b=trial.suggest_float('b',0, 10)


    losses=[]
    for batch,(data, target,idx) in enumerate(valid_dl):
        outputs = (a*model_0(data)+b*model_1(data))/(a+b)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    # print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)



    return loss

study_en = optuna.create_study(direction='minimize')

study_en.optimize(objective_en, n_trials=25)

In [959]:
a=study_en.best_params['a']
b=study_en.best_params['b']

In [960]:
model_0.eval()
output=[]
for i in range(len(testset)):
    output.append(((a*model_0(testset.x[i])+b*model_1(testset.x[i]))/(a+b)).item())

sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)

In [939]:
model_0.eval()

for batch,(data, target,idx) in enumerate(valid_dl):
    outputs = model_0(data)
    loss = criterion(outputs.reshape(-1), target.reshape(-1))
print(f"loss: {loss.item():>7f} ")

loss: 0.026304 


In [940]:
model_0.eval()
indexF=[]
indexT=[]
for i in range(len(trainset)):
    outputs = model_0(trainset.dataset.x[i])
    loss = criterion(outputs,trainset.dataset.y[i].reshape(-1))
    if(loss>0.05):
        indexF.append(i)
    else:
        indexT.append(i)

In [941]:
Fdataset=torch.utils.data.Subset(trainset, indexF)
Tdataset=torch.utils.data.Subset(trainset, indexT)
Fdl=DataLoader(Fdataset, shuffle=True, batch_size=batch_size)
Tdl=DataLoader(Tdataset, shuffle=True, batch_size=batch_size)

In [942]:
len(Fdl.dataset)

950

In [944]:
def objective(trial):
        
        params = {

            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
            'batch_size': trial.suggest_int("batch_size", 64, 320,step=16),
            'num_hidden1': trial.suggest_int("num_hidden1", 50, 500),
            'num_hidden2': trial.suggest_int("num_hidden2", 50, 600),
            'num_hidden3': trial.suggest_int("num_hidden3", 50, 600),
            'num_hidden4': trial.suggest_int("num_hidden4", 50, 600),
            'num_hidden5': trial.suggest_int("num_hidden5", 50, 600),
            'num_hidden6': trial.suggest_int("num_hidden6", 50, 600),
            'threshold':trial.suggest_float('threshold', 1e-3, 0.5),
            }
        

        train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
        valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])

        model_0.eval()
        
        indexT=[]
        for i in range(len(trainset)):
            outputs = model_0(trainset.dataset.x[i])
            loss = criterion(outputs,trainset.dataset.y[i].reshape(-1))
            if(loss<params['threshold']):
                indexT.append(i)
        
        Tdataset=torch.utils.data.Subset(trainset, indexT)
        
        Tdl=DataLoader(Tdataset, shuffle=True, batch_size=params['batch_size'])
      
        model= MLP(params)
        best_model,loss=train_and_evaluate(params,model,Tdl)
      
        torch.save(best_model.state_dict(), f"model_trial_{trial.number}.pth")
    
        return loss
    

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

[I 2023-05-02 19:01:29,467] A new study created in memory with name: no-name-87e75915-82ad-4ea8-b747-7b1489600129
[I 2023-05-02 19:01:43,184] Trial 0 finished with value: 0.058144922123143546 and parameters: {'learning_rate': 0.05374124154482045, 'optimizer': 'SGD', 'batch_size': 192, 'num_hidden1': 214, 'num_hidden2': 582, 'num_hidden3': 578, 'num_hidden4': 205, 'num_hidden5': 577, 'num_hidden6': 84, 'threshold': 0.42203550238953014}. Best is trial 0 with value: 0.058144922123143546.


loss: 0.058145 


[I 2023-05-02 19:02:03,325] Trial 1 finished with value: 0.1733101440424269 and parameters: {'learning_rate': 0.05947035732137346, 'optimizer': 'Adam', 'batch_size': 160, 'num_hidden1': 270, 'num_hidden2': 522, 'num_hidden3': 475, 'num_hidden4': 52, 'num_hidden5': 228, 'num_hidden6': 197, 'threshold': 0.09424444331087886}. Best is trial 0 with value: 0.058144922123143546.


loss: 0.173310 


[I 2023-05-02 19:02:27,387] Trial 2 finished with value: 0.03079753652215004 and parameters: {'learning_rate': 0.005117201830603615, 'optimizer': 'Adam', 'batch_size': 144, 'num_hidden1': 159, 'num_hidden2': 595, 'num_hidden3': 297, 'num_hidden4': 234, 'num_hidden5': 354, 'num_hidden6': 372, 'threshold': 0.4924604709576808}. Best is trial 2 with value: 0.03079753652215004.


...

loss: 0.025702 


[I 2023-05-02 19:30:14,077] Trial 98 finished with value: 0.029209880717098714 and parameters: {'learning_rate': 0.00771988720231155, 'optimizer': 'Adam', 'batch_size': 80, 'num_hidden1': 322, 'num_hidden2': 436, 'num_hidden3': 471, 'num_hidden4': 265, 'num_hidden5': 233, 'num_hidden6': 543, 'threshold': 0.4016266943172981}. Best is trial 85 with value: 0.022372110605691418.


loss: 0.029210 


[I 2023-05-02 19:30:39,258] Trial 99 finished with value: 0.10754392183188236 and parameters: {'learning_rate': 0.01381309236777491, 'optimizer': 'Adam', 'batch_size': 224, 'num_hidden1': 298, 'num_hidden2': 409, 'num_hidden3': 506, 'num_hidden4': 420, 'num_hidden5': 413, 'num_hidden6': 590, 'threshold': 0.43712857545723466}. Best is trial 85 with value: 0.022372110605691418.


loss: 0.107544 


In [ ]:
# display(optuna.visualization.plot_param_importances(study))
# display(optuna.visualization.plot_parallel_coordinate(study))
# display(optuna.visualization.plot_optimization_history(study))

In [945]:
model_op= MLP(study.best_params)
model_op.load_state_dict(torch.load("C:/Users/MJ/Desktop/FunctionApproximation/model_trial_85.pth"))
model_op.eval()
losses=[]
for batch,(data, target,idx) in enumerate(valid_dl):
    outputs = model_op(data)
    loss = criterion(outputs.reshape(-1), target.reshape(-1))
    losses.append(loss.item())
print(f"loss: {sum(losses)/len(losses):>7f} ")
loss=sum(losses)/len(losses)



loss: 0.023143 


In [ ]:
# def objective_en(trial):
#     a=trial.suggest_float('a',0, 10)
#     b=trial.suggest_float('b',0, 10)


#     losses=[]
#     for batch,(data, target,idx) in enumerate(valid_dl):
#         outputs = (a*model_op(data)+b*model_sec(data))/(a+b)
#         loss = criterion(outputs.reshape(-1), target.reshape(-1))
#         losses.append(loss.item())
#     print(f"loss: {sum(losses)/len(losses):>7f} ")
#     loss=sum(losses)/len(losses)



#     return loss

# study_en = optuna.create_study(direction='minimize')
# optuna.logging.set_verbosity(optuna.logging.WARNING)
# study_en.optimize(objective_en, n_trials=25)

In [946]:
model_op.eval()
output=[]
for i in range(len(testset)):
    output.append((model_op(testset.x[i])).item())

sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)